В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [66]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier

from modules.modules_2.topic_2_3.homework.process_bank_churn import preprocess_data, preprocess_new_data

In [84]:
raw_df = pd.read_csv('../../../../bank-customer-churn-prediction-dlu/data/train.csv', index_col=0)

preprocessed_data = preprocess_data(raw_df, scaler_numeric=False)

X_train = preprocessed_data['X_train']
train_targets = preprocessed_data['train_targets']
X_val = preprocessed_data['X_val']
val_targets = preprocessed_data['val_targets']
scaler = preprocessed_data['scaler']
encoder = preprocessed_data['encoder']

Numeric columns: ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
Binary columns: ['IsActiveMember', 'HasCrCard']
Categorical columns: ['Geography', 'Gender']


1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [47]:
# By default it has 5 neighbors
knn_base = KNeighborsClassifier()

# Fit model with train data
knn_base.fit(X_train, train_targets)

# Predict on dataset which model has not seen before
knn_base.predict(X_val)

array([0., 0., 0., ..., 0., 0., 0.])

In [48]:
# Calculate AUROC for train and validation datasets
train_auc = roc_auc_score(train_targets, knn_base.predict_proba(X_train)[:,1])
val_auc = roc_auc_score(val_targets, knn_base.predict_proba(X_val)[:,1])

train_auc, val_auc

(np.float64(0.9559428457733543), np.float64(0.852585568283147))

**Спостореження:**
Модель виглядає перенавчена (high variance), оскільки має високий показник на тренувальній вибірці, але низький на валідаційній.

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [49]:
_knn = KNeighborsClassifier()
params_knn = {'n_neighbors': range(1, 50)}
# NOTE: GridSearchCV має scoring куди можна додати 'roc_auc'
knn_gs = GridSearchCV(_knn, params_knn, cv=5)
knn_gs.fit(X_train, train_targets)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 50)})

In [50]:
knn_best = knn_gs.best_estimator_

print(knn_gs.best_params_, knn_gs.best_score_)

{'n_neighbors': 9} 0.8671666666666666


In [51]:
train_auc = roc_auc_score(train_targets, knn_best.predict_proba(X_train)[:,1])
val_auc = roc_auc_score(val_targets, knn_best.predict_proba(X_val)[:,1])

train_auc, val_auc

(np.float64(0.9407278147861952), np.float64(0.8808731737430551))

**Спостереження:**
Ця модель у порівнянні із попередньою трохи краще на валідаційних, але досі вона перенавчена. Ми бачимо, що на тренувальних даних маємо 0.94, а на валідаційних - 0.88. Але поки виглядає, що поки що перенавчена.

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [54]:
RANDOM_STATE = 42

dt = DecisionTreeClassifier(random_state=RANDOM_STATE)

params_dt = {
    'max_depth': np.arange(1, 20, 2),
    'max_leaf_nodes': np.arange(2, 10)
}

dt_gs = GridSearchCV(dt, params_dt, cv=3, scoring='roc_auc')

dt_gs.fit(X_train, train_targets)

print(dt_gs.best_params_, dt_gs.best_score_)
dt_best = dt_gs.best_estimator_

train_auc = roc_auc_score(train_targets, dt_best.predict_proba(X_train)[:,1])
val_auc = roc_auc_score(val_targets, dt_best.predict_proba(X_val)[:,1])

print(f'{train_auc=}, {val_auc=}')

{'max_depth': np.int64(5), 'max_leaf_nodes': np.int64(9)} 0.8990230600400091
train_auc=np.float64(0.9001138615074584), val_auc=np.float64(0.898395980519926)


**Спостереження:**
Ця модель виглядає як релевантана для продакшину так як має дуже хороші показники і вона не перенавчена і не недонавчена.
Показники на тренувальних та на валідаційних однакові. Ця модель однозначно краще за усі попередні.

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [56]:
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

In [62]:
%%time
dt = DecisionTreeClassifier(random_state=RANDOM_STATE)

dt_rs = RandomizedSearchCV(dt, params_dt, n_iter=40, cv=3, scoring='roc_auc', random_state=RANDOM_STATE)

dt_rs.fit(X_train, train_targets)

print(f'{dt_rs.best_params_=}')
print(f'{dt_rs.best_score_=}')
dt_random_search_best = dt_rs.best_estimator_

dt_rs.best_params_={'splitter': 'best', 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_leaf_nodes': np.int64(14), 'max_features': None, 'max_depth': np.int64(16), 'criterion': 'entropy'}
dt_rs.best_score_=np.float64(0.910864318350194)
CPU times: user 526 ms, sys: 12.5 ms, total: 538 ms
Wall time: 538 ms


In [63]:
train_auc = roc_auc_score(train_targets, dt_random_search_best.predict_proba(X_train)[:,1])
val_auc = roc_auc_score(val_targets, dt_random_search_best.predict_proba(X_val)[:,1])

print(f'{train_auc=}, {val_auc=}')

train_auc=np.float64(0.9169275635848141), val_auc=np.float64(0.9166204815145071)


**Спостереження:**
Ця модель є ще кращою від попередньої. Вона має кращі показники на тренувальних та валідаційних даних на ~0.01 пункт.

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [95]:
%%time
experiment_params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 25),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20, 25, 30, 35],
    'min_samples_leaf': [2, 3, 4, 6, 8],
    'max_features': [None, 'sqrt', 'log2']
}

dt = DecisionTreeClassifier(random_state=RANDOM_STATE)

dt_rs = RandomizedSearchCV(dt, params_dt, n_iter=400, cv=10, scoring='roc_auc', random_state=RANDOM_STATE)

dt_rs.fit(X_train, train_targets)

print(f'{dt_rs.best_params_=}')
print(f'{dt_rs.best_score_=}')
dt_random_search_best = dt_rs.best_estimator_

train_auc = roc_auc_score(train_targets, dt_random_search_best.predict_proba(X_train)[:,1])
val_auc = roc_auc_score(val_targets, dt_random_search_best.predict_proba(X_val)[:,1])

print(f'{train_auc=}, {val_auc=}')

dt_rs.best_params_={'splitter': 'best', 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_leaf_nodes': np.int64(19), 'max_features': None, 'max_depth': np.int64(9), 'criterion': 'entropy'}
dt_rs.best_score_=np.float64(0.9173135345374055)
train_auc=np.float64(0.9214076043172088), val_auc=np.float64(0.9220214006447629)
CPU times: user 19.2 s, sys: 416 ms, total: 19.6 s
Wall time: 19.7 s


In [91]:
test_raw_df = pd.read_csv('../../topic_2_2/homeworks/bank-customer-churn-prediction-dlu/test.csv')
sample_submission = pd.read_csv('../../topic_2_2/homeworks/bank-customer-churn-prediction-dlu/sample_submission.csv')

preprocessed_test_df = preprocess_new_data(test_raw_df, scaler, encoder, scaler_numeric=False)

Numeric columns: ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
Binary columns: ['IsActiveMember', 'HasCrCard']
Categorical columns: ['Geography', 'Gender']


In [86]:
preprocessed_test_df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Male
0,584.0,30.0,2.0,146053.66,1.0,1.0,1.0,157891.86,0.0,1.0,0.0,1.0
1,551.0,39.0,5.0,0.00,2.0,1.0,1.0,67431.28,1.0,0.0,0.0,1.0
2,706.0,43.0,8.0,0.00,2.0,1.0,0.0,156768.45,1.0,0.0,0.0,1.0
3,717.0,45.0,3.0,0.00,1.0,1.0,1.0,166909.87,0.0,0.0,1.0,1.0
4,592.0,43.0,8.0,0.00,2.0,1.0,1.0,143681.97,0.0,0.0,1.0,1.0


In [94]:
test_preds = dt_random_search_best.predict_proba(preprocessed_test_df)[:, 1]

print(test_preds)
sample_submission['Exited'] = test_preds

# Save the submission
sample_submission.to_csv(f'submission_decision_trees_max_depth_{dt_random_search_best.get_depth()}_max_leaf_nodes_{dt_random_search_best.get_n_leaves()}.csv', index=False)

[0.23791103 0.01211499 0.14507772 ... 0.01211499 0.03930131 0.18763797]
